Import the librairies and modules:

In [1]:
import numpy as np
import networkx as nx
import os
import csv
from BAG_Code_tw520.createORtable import create_OR_table
from BAG_Code_tw520.BayesianAttackGraphForCVSS import parse_dot
from Threat_Inteligence.CWE_tree.parse import parse_xml
from BAG_Code_tw520.BayesianAttackGraphForCVSS import change_prob
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference.ExactInference import BeliefPropagation

To prepare the simulation use the following to put the graph in memory:

In [2]:
# Name of the simulation
simulation = "OCD_pentest"

# Path to the folder containing the tree
path = "C:/Users/docuser/Documents/ImperialWork/Personnal_simulations/output_" + simulation + "/strongly_connected_components/"
file_name = "ag-nocycles.dot"
path_to_dot = path + file_name
output_file = path + file_name[:-4] + "_inference.txt"

# We all read from the file, adding probabilities in the same time
BAG, edges, nodes = parse_dot(open(path_to_dot, 'r').read())

# This is the reference BAG, before the attacker has compromised any node
BAG_ref = BAG.copy()
prop0 = BeliefPropagation(BAG_ref)

# We create a dictionary to get the node number from the label
inverted_nodes = {int(v['label'].split(':')[0]): k for k, v in nodes.items()}

In [1]:
# Introducing link between CWEs themselves
Gcwe = nx.Graph()
parse_xml(os.getcwd()+'/Threat_Inteligence/CWE_tree/1000.xml', Gcwe)

# Introducing link between CWEs and CVEs
cwe_file = csv.DictReader(open(os.getcwd()+'/Threat_Inteligence/CVE_to_CWE.csv', 'r'))
cwe_dict = {row['Vulnerability']: row['weakness'] for row in cwe_file}

NameError: name 'nx' is not defined

# When the attacker become admin of host and we want to recalculate for domain admin
src_node = [31, 20, 23, 25, 27, 29]
dst_node = [2,7, 9, 11, 15, 17]

In [8]:
# We list the CVE that are before the attacker position
src_node = [7, 10, 12, 21, 25, 27]
dst_node = [ 14, 16, 18]
# We always start from the original probabilities of the CVEs
BAG = BAG_ref.copy()

# We modify the conditional probabilities according to the attacks that have been done
change_prob(BAG, edges, nodes, src_node, dst_node, inverted_nodes, cwe_dict, Gcwe, prop0)

CVE-2021-42278
CVE-2021-42287
CVE-2022-26923


In [9]:
#We use the pgmpy library to perform the inference
prop = BeliefPropagation(BAG)

cibles = [1]
for cible in cibles:
    print(prop0.query([inverted_nodes[cible]]))
    print(prop0.query([inverted_nodes[cible]], evidence={6:1}))
    print(prop.query([inverted_nodes[cible]], evidence={6:1}))

+------+----------+
| 1    |   phi(1) |
+======+==========+
| 1(0) |   0.5220 |
+------+----------+
| 1(1) |   0.4780 |
+------+----------+
+------+----------+
| 1    |   phi(1) |
+======+==========+
| 1(0) |   0.1603 |
+------+----------+
| 1(1) |   0.8397 |
+------+----------+
+------+----------+
| 1    |   phi(1) |
+======+==========+
| 1(0) |   0.1600 |
+------+----------+
| 1(1) |   0.8400 |
+------+----------+


In [ ]:
# distance = nx.shortest_path_length(Gcwe, source='119', target='255')
# print("Distance between nodes '1004' and '732':", distance)

# print(BAG.get_cpds(inverted_nodes[7]))
# table = create_OR_table([0.1])
# source = []
# for edge in edges:
#     if edge[1] == inverted_nodes[7]:
#         source.append(edge[0])
# npa = len(source)
# BAG.add_cpds(TabularCPD(inverted_nodes[7], 2, table.T, source, evidence_card=2*np.ones(npa)))
# print(BAG.get_cpds(inverted_nodes[7]))


# output = open(output_file, 'w')
# for node in nodes.items():
#     output.write(f'{nodes[node[0]]["label"]} : {prop.query([node[0]])}\n')